In [1]:
# @hidden_cell


# This connection object is used to access your data and contains your credentials or project token.
# You might want to remove those credentials before you share your notebook.

import os
import types
import pandas as pd
import ibm_boto3
from botocore.client import Config
import logging

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.

connection_TMS_trafficData_client = ibm_boto3.client(
    service_name='s3',
    ibm_api_key_id='MLD98EDcdqsLiCo_5wWSJT62E1LB_z61DyFwrlKK5Pqf',
    ibm_service_instance_id='crn:v1:bluemix:public:cloud-object-storage:global:a/7e7b7ae0d80b4f33b4e078bb2650033c:8a835f62-b295-423a-8d61-9b0d9819f636::',
    ibm_auth_endpoint='https://iam.cloud.ibm.com/identity/token',
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-de.cloud-object-storage.appdomain.cloud'
)

body = connection_TMS_trafficData_client.get_object(Bucket='smart-city-tms', Key='topics/digitraffic_tms/partition=0/digitraffic_tms+0+0011067000.json')['Body'].read()
# add missing __iter__ method, so pandas accepts body as file-like object 

if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face an error during data loading.
# Refer to the documentation of 'pandas.read_json()' and 'pandas.io.json.json_normalize' for more possibilities to adjust the data loading.
# pandas documentation: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
# and http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html

pd.read_json(body, lines=True)

,measuredTime,roadStationId,oldName,name,sensorUnit,id,shortName,sensorValue,timeWindowStart,timeWindowEnd,lastUpdated,lastError,type,status
0,2022-02-12T05:50:06Z,23188.0,keskinopeus_5min_liukuva_suunta1_VVAPAAS1,KESKINOPEUS_5MIN_LIUKUVA_SUUNTA1_VVAPAAS1,***,5158.0,LTila1,114.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-02-12T05:50:06Z,23188.0,ohitukset_5min_liukuva_suunta2_MS2,OHITUKSET_5MIN_LIUKUVA_SUUNTA2_MS2,***,5168.0,MTila2,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-02-12T05:50:06Z,23188.0,ohitukset_60min_kiintea_suunta2,OHITUKSET_60MIN_KIINTEA_SUUNTA2,kpl/h,5055.0,kpl/h2,37.0,2022-02-12T04:00:00Z,2022-02-12T05:00:00Z,NaN,NaN,NaN,NaN
3,2022-02-12T05:50:06Z,23188.0,trafficVolume2,OHITUKSET_5MIN_LIUKUVA_SUUNTA2,kpl/h,5119.0,kpl/h2,36.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-02-12T05:50:06Z,23188.0,keskinopeus_5min_kiintea_suunta2_VVAPAAS2,KESKINOPEUS_5MIN_KIINTEA_SUUNTA2_VVAPAAS2,***,5061.0,LTil2,99.0,2022-02-12T05:45:00Z,2022-02-12T05:50:00Z,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2022-02-12T05:50:49Z,23309.0,ohitukset_60min_kiintea_suunta1_MS1,OHITUKSET_60MIN_KIINTEA_SUUNTA1_MS1,***,5067.0,MTil1,2.0,2022-02-12T04:00:00Z,2022-02-12T05:00:00Z,NaN,NaN,NaN,NaN
4996,2022-02-12T05:50:48Z,23309.0,trafficVolume2,OHITUKSET_5MIN_LIUKUVA_SUUNTA2,kpl/h,5119.0,kpl/h2,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4997,2022-02-12T05:50:49Z,23309.0,ohitukset_60min_kiintea_suunta1,OHITUKSET_60MIN_KIINTEA_SUUNTA1,kpl/h,5054.0,kpl/h1,29.0,2022-02-12T04:00:00Z,2022-02-12T05:00:00Z,NaN,NaN,NaN,NaN
4998,2022-02-12T05:50:49Z,23309.0,averageSpeed1,KESKINOPEUS_5MIN_LIUKUVA_SUUNTA1,km/h,5122.0,km/h1,87.0,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
import argparse
import boto3
import dateutil.parser
import logging
import pytz
from collections import namedtuple

import pandas as pd
from datetime import datetime, timezone, timedelta



logger = logging.getLogger(__name__)


Rule = namedtuple('Rule', ['has_min', 'has_max'])
last_modified_rules = {
    Rule(has_min=True, has_max=True):
        lambda min_date, date, max_date: min_date <= date <= max_date,
    Rule(has_min=True, has_max=False):
        lambda min_date, date, max_date: min_date <= date,
    Rule(has_min=False, has_max=True):
        lambda min_date, date, max_date: date <= max_date,
    Rule(has_min=False, has_max=False):
        lambda min_date, date, max_date: True,
}

def get_s3_objects(s3, bucket, prefixes=None, suffixes=None, last_modified_min=None, last_modified_max=None):
    """
    Generate the objects in an S3 bucket. Adapted from:
    https://alexwlchan.net/2017/07/listing-s3-keys/

    :param bucket: Name of the S3 bucket.
    :ptype bucket: str
    :param prefixes: Only fetch keys that start with these prefixes (optional).
    :ptype prefixes: tuple
    :param suffixes: Only fetch keys that end with thes suffixes (optional).
    :ptype suffixes: tuple
    :param last_modified_min: Only yield objects with LastModified dates greater than this value (optional).
    :ptype last_modified_min: datetime.date
    :param last_modified_max: Only yield objects with LastModified dates greater than this value (optional).
    :ptype last_modified_max: datetime.date

    :returns: generator of dictionary objects
    :rtype: dict https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.list_objects
    """
    if last_modified_min and last_modified_max and last_modified_max < last_modified_min:
        raise ValueError(
            "When using both, last_modified_max: {} must be greater than last_modified_min: {}".format(
                last_modified_max, last_modified_min
            )
        )
    # Use the last_modified_rules dict to lookup which conditional logic to apply
    # based on which arguments were supplied
    last_modified_rule = last_modified_rules[bool(last_modified_min), bool(last_modified_max)]

    if not prefixes:
        prefixes = ('',)
    else:
        prefixes = tuple(set(prefixes))
    if not suffixes:
        suffixes = ('',)
    else:
        suffixes = tuple(set(suffixes))

    kwargs = {'Bucket': bucket}

    for prefix in prefixes:
        kwargs['Prefix'] = prefix
        while True:
            # The S3 API response is a large blob of metadata.
            # 'Contents' contains information about the listed objects.
            resp = s3.list_objects_v2(**kwargs)
            for content in resp.get('Contents', []):
                last_modified_date = content['LastModified']
                if (
                    content['Key'].endswith(suffixes) and
                    last_modified_rule(last_modified_min, last_modified_date, last_modified_max)
                ):
                    yield content

            # The S3 API is paginated, returning up to 1000 keys at a time.
            # Pass the continuation token into the next response, until we
            # reach the final page (when this field is missing).
            try:
                kwargs['ContinuationToken'] = resp['NextContinuationToken']
            except KeyError:
                break

In [3]:
dateloading = "2022-02-25"
#dateloading = str(os.environ['DATE'])
starttime = datetime.fromisoformat(dateloading + ' 00:00:00.000+00:00')
endtime = datetime.fromisoformat(dateloading + ' 23:59:59.999+00:00')
metastarttime = datetime.fromisoformat("2022-02-20" + ' 00:00:00.000+00:00')
filename_starttime = starttime.hour
filename_endtime = endtime.hour

In [4]:
#hoursago = datetime.now(timezone.utc) - timedelta(hours=200, minutes=0)
#hours = datetime.now(timezone.utc) - timedelta(hours=176, minutes=0)
objs=get_s3_objects(s3=connection_TMS_trafficData_client,bucket="smart-city-tms", last_modified_min=starttime, last_modified_max=endtime)

In [5]:
df_tms = pd.DataFrame()
for obj in objs:
    df_tms = df_tms.append(pd.read_json(connection_TMS_trafficData_client.get_object(Bucket='smart-city-tms', Key=obj['Key'])['Body'].read(), lines=True))
    
df_tms = df_tms.reset_index()
df_tms = df_tms[df_tms['roadStationId'].notna()]

In [6]:
df_tms.shape

(780280, 15)

In [7]:
logging.info(df_tms.shape)

## Datumsformat anpassen

In [8]:
import datetime

date = []

for x in df_tms.measuredTime:
    x = x[:-7]
    date_obj = datetime.datetime.strptime(x, '%Y-%m-%dT%H')
    date.append(str(date_obj.date()) + " " + str(date_obj.time()))


df_tms["timestamp"] = date

In [9]:
df_tms.head()

,index,measuredTime,roadStationId,oldName,name,sensorUnit,id,shortName,sensorValue,timeWindowStart,timeWindowEnd,lastUpdated,lastError,type,status,timestamp
0,0,2022-03-12T23:59:35Z,23575.0,keskinopeus_5min_liukuva_suunta1_VVAPAAS1,KESKINOPEUS_5MIN_LIUKUVA_SUUNTA1_VVAPAAS1,***,5158.0,LTila1,139.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-12 23:00:00
1,1,2022-03-12T23:59:35Z,23575.0,ohitukset_5min_liukuva_suunta1_MS1,OHITUKSET_5MIN_LIUKUVA_SUUNTA1_MS1,***,5164.0,MTila1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-12 23:00:00
2,2,2022-03-12T23:59:35Z,23575.0,ohitukset_60min_kiintea_suunta2_MS2,OHITUKSET_60MIN_KIINTEA_SUUNTA2_MS2,***,5071.0,MTil2,1.0,2022-03-12T22:00:00Z,2022-03-12T23:00:00Z,NaN,NaN,NaN,NaN,2022-03-12 23:00:00
3,3,2022-03-12T23:59:35Z,23575.0,keskinopeus_60min_kiintea_suunta2,KESKINOPEUS_60MIN_KIINTEA_SUUNTA2,km/h,5057.0,km/h2,103.0,2022-03-12T22:00:00Z,2022-03-12T23:00:00Z,NaN,NaN,NaN,NaN,2022-03-12 23:00:00
4,4,2022-03-12T23:59:35Z,23575.0,ohitukset_5min_liukuva_suunta2_MS2,OHITUKSET_5MIN_LIUKUVA_SUUNTA2_MS2,***,5168.0,MTila2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-12 23:00:00


## Duplikate über roadStationId, id und timestamp droppen

In [10]:
df_tms = df_tms.drop_duplicates(subset=['roadStationId', 'id', 'timestamp'])

In [11]:
df_tms.shape

(18924, 16)

In [12]:
df_tms.head()

,index,measuredTime,roadStationId,oldName,name,sensorUnit,id,shortName,sensorValue,timeWindowStart,timeWindowEnd,lastUpdated,lastError,type,status,timestamp
0,0,2022-03-12T23:59:35Z,23575.0,keskinopeus_5min_liukuva_suunta1_VVAPAAS1,KESKINOPEUS_5MIN_LIUKUVA_SUUNTA1_VVAPAAS1,***,5158.0,LTila1,139.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-12 23:00:00
1,1,2022-03-12T23:59:35Z,23575.0,ohitukset_5min_liukuva_suunta1_MS1,OHITUKSET_5MIN_LIUKUVA_SUUNTA1_MS1,***,5164.0,MTila1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-12 23:00:00
2,2,2022-03-12T23:59:35Z,23575.0,ohitukset_60min_kiintea_suunta2_MS2,OHITUKSET_60MIN_KIINTEA_SUUNTA2_MS2,***,5071.0,MTil2,1.0,2022-03-12T22:00:00Z,2022-03-12T23:00:00Z,NaN,NaN,NaN,NaN,2022-03-12 23:00:00
3,3,2022-03-12T23:59:35Z,23575.0,keskinopeus_60min_kiintea_suunta2,KESKINOPEUS_60MIN_KIINTEA_SUUNTA2,km/h,5057.0,km/h2,103.0,2022-03-12T22:00:00Z,2022-03-12T23:00:00Z,NaN,NaN,NaN,NaN,2022-03-12 23:00:00
4,4,2022-03-12T23:59:35Z,23575.0,ohitukset_5min_liukuva_suunta2_MS2,OHITUKSET_5MIN_LIUKUVA_SUUNTA2_MS2,***,5168.0,MTila2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-12 23:00:00


# Metadaten laden


In [13]:
#metadaten der TMS Stations
objs_tms_meta=get_s3_objects(s3=connection_TMS_trafficData_client,bucket="smart-city-tms-stations-metadata", last_modified_min=metastarttime, last_modified_max=endtime)
df_tms_meta = pd.DataFrame()

for obj in objs_tms_meta:
    if df_tms_meta.empty:
        df_tms_meta = pd.read_json(connection_TMS_trafficData_client.get_object(Bucket='smart-city-tms-stations-metadata', Key=obj['Key'])['Body'].read(), lines=True)
    else:
        df_tms_meta_tmp = pd.read_json(connection_TMS_trafficData_client.get_object(Bucket='smart-city-tms-stations-metadata', Key=obj['Key'])['Body'].read(), lines=True)
        df_tms_meta = df_tms_meta.append(df_tms_meta_tmp)
df_tms_meta = df_tms_meta.reset_index()

## Metadaten transformieren

In [14]:
#Datentransformation
df = pd.json_normalize(df_tms_meta['geometry'])
print(df)
df = pd.DataFrame(df.iloc[:,0].tolist(), columns=['longitude', 'latitude', 'else'])
df = df[['latitude', 'longitude']]
df2 = df_tms_meta['id']
df2 = df2.reset_index()
df = df.reset_index()
df = df.join(df2.set_index('index'), on='index', how='inner')
df = df[['latitude', 'longitude', 'id']]
df = df.rename(columns={'id': 'roadStationId'})
df = df.drop_duplicates(subset=['roadStationId'])
df.dropna()

                       coordinates   type
0      [25.689529, 60.417002, 0.0]  Point
1      [24.664459, 60.153502, 0.0]  Point
2           [24.849501, 60.3, 0.0]  Point
3      [24.808791, 60.354656, 0.0]  Point
4      [24.746882, 60.211601, 0.0]  Point
...                            ...    ...
15295  [25.508377, 65.019337, 0.0]  Point
15296  [25.462881, 65.046788, 0.0]  Point
15297   [25.49017, 64.949734, 0.0]  Point
15298  [25.505382, 64.842413, 0.0]  Point
15299  [25.494083, 65.032306, 0.0]  Point

[15300 rows x 2 columns]


,latitude,longitude,roadStationId
0,60.417002,25.689529,23001
1,60.153502,24.664459,23003
2,60.300000,24.849501,23004
3,60.354656,24.808791,23005
4,60.211601,24.746882,23006
...,...,...,...
495,60.325108,23.949030,24606
496,60.289063,24.079464,24607
4882,61.898973,27.823412,20601
4883,61.899017,27.823508,20602


## Metadaten auf die TMS Daten joinen

In [15]:
df_tms = pd.merge(df_tms, df, left_on='roadStationId', right_on='roadStationId', how='left')

In [16]:
df_tms.shape

(18924, 18)

In [17]:
df_tms.head()

,index,measuredTime,roadStationId,oldName,name,sensorUnit,id,shortName,sensorValue,timeWindowStart,timeWindowEnd,lastUpdated,lastError,type,status,timestamp,latitude,longitude
0,0,2022-03-12T23:59:35Z,23575.0,keskinopeus_5min_liukuva_suunta1_VVAPAAS1,KESKINOPEUS_5MIN_LIUKUVA_SUUNTA1_VVAPAAS1,***,5158.0,LTila1,139.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-12 23:00:00,60.486397,26.54646
1,1,2022-03-12T23:59:35Z,23575.0,ohitukset_5min_liukuva_suunta1_MS1,OHITUKSET_5MIN_LIUKUVA_SUUNTA1_MS1,***,5164.0,MTila1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-12 23:00:00,60.486397,26.54646
2,2,2022-03-12T23:59:35Z,23575.0,ohitukset_60min_kiintea_suunta2_MS2,OHITUKSET_60MIN_KIINTEA_SUUNTA2_MS2,***,5071.0,MTil2,1.0,2022-03-12T22:00:00Z,2022-03-12T23:00:00Z,NaN,NaN,NaN,NaN,2022-03-12 23:00:00,60.486397,26.54646
3,3,2022-03-12T23:59:35Z,23575.0,keskinopeus_60min_kiintea_suunta2,KESKINOPEUS_60MIN_KIINTEA_SUUNTA2,km/h,5057.0,km/h2,103.0,2022-03-12T22:00:00Z,2022-03-12T23:00:00Z,NaN,NaN,NaN,NaN,2022-03-12 23:00:00,60.486397,26.54646
4,4,2022-03-12T23:59:35Z,23575.0,ohitukset_5min_liukuva_suunta2_MS2,OHITUKSET_5MIN_LIUKUVA_SUUNTA2_MS2,***,5168.0,MTila2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-12 23:00:00,60.486397,26.54646


# TMS Daten cleanen

## Irrelevanten Features löschen

Zu löschen sind:
- oldName
- sensorUnit
- shortName
- lastUpdated
- lastError
- type
- status
- timeWindowStart
- timeWindowEnd

In [18]:
df_tms_clean = df_tms.drop(['name', 'timeWindowStart', 'timeWindowEnd', 'oldName', 'sensorUnit', 'shortName', 'lastUpdated', 'lastError', 'type', 'status', 'index'], axis=1)

In [19]:
df_tms_clean.head()

,measuredTime,roadStationId,id,sensorValue,timestamp,latitude,longitude
0,2022-03-12T23:59:35Z,23575.0,5158.0,139.0,2022-03-12 23:00:00,60.486397,26.54646
1,2022-03-12T23:59:35Z,23575.0,5164.0,1.0,2022-03-12 23:00:00,60.486397,26.54646
2,2022-03-12T23:59:35Z,23575.0,5071.0,1.0,2022-03-12 23:00:00,60.486397,26.54646
3,2022-03-12T23:59:35Z,23575.0,5057.0,103.0,2022-03-12 23:00:00,60.486397,26.54646
4,2022-03-12T23:59:35Z,23575.0,5168.0,1.0,2022-03-12 23:00:00,60.486397,26.54646


## Nach Stunden Daten sortieren
(https://www.digitraffic.fi/en/road-traffic/lam/)

Relevanten IDs: 
- 5056
- 5057
- 5054
- 5055
- 5067
- 5071

In [20]:
df_tms_hour = df_tms_clean[df_tms_clean.id.isin([5056, 5057, 5054, 5055, 5067, 5071])]

In [21]:
df_tms_hour.head()

,measuredTime,roadStationId,id,sensorValue,timestamp,latitude,longitude
2,2022-03-12T23:59:35Z,23575.0,5071.0,1.0,2022-03-12 23:00:00,60.486397,26.546460
3,2022-03-12T23:59:35Z,23575.0,5057.0,103.0,2022-03-12 23:00:00,60.486397,26.546460
7,2022-03-12T23:59:35Z,23575.0,5067.0,2.0,2022-03-12 23:00:00,60.486397,26.546460
13,2022-03-12T23:59:45Z,23576.0,5054.0,69.0,2022-03-12 23:00:00,60.514416,26.926898
17,2022-03-12T23:59:45Z,23576.0,5071.0,3.0,2022-03-12 23:00:00,60.514416,26.926898


In [22]:
df_tms_hour = df_tms_hour.reset_index(drop=True)

In [23]:
df_tms_hour.head()

,measuredTime,roadStationId,id,sensorValue,timestamp,latitude,longitude
0,2022-03-12T23:59:35Z,23575.0,5071.0,1.0,2022-03-12 23:00:00,60.486397,26.546460
1,2022-03-12T23:59:35Z,23575.0,5057.0,103.0,2022-03-12 23:00:00,60.486397,26.546460
2,2022-03-12T23:59:35Z,23575.0,5067.0,2.0,2022-03-12 23:00:00,60.486397,26.546460
3,2022-03-12T23:59:45Z,23576.0,5054.0,69.0,2022-03-12 23:00:00,60.514416,26.926898
4,2022-03-12T23:59:45Z,23576.0,5071.0,3.0,2022-03-12 23:00:00,60.514416,26.926898


## Abstand zu unserem zentralen Geopunkt (20km Radius zum Zentrum von Helsinki)

In [24]:
import geopy
from geopy import distance
#(Lat, Long)!!
#fix ~ middle of Helsinki

fix = (60.192059, 24.945831)
abstand = []
for x in range(len(df_tms_hour)):
    distance = geopy.distance.distance((df_tms_hour.latitude[x], df_tms_hour.longitude[x]), fix).km
    abstand.append(distance)

df_tms_hour['distance'] = abstand
df_tms_hour

,measuredTime,roadStationId,id,sensorValue,timestamp,latitude,longitude,distance
0,2022-03-12T23:59:35Z,23575.0,5071.0,1.0,2022-03-12 23:00:00,60.486397,26.546460,94.283255
1,2022-03-12T23:59:35Z,23575.0,5057.0,103.0,2022-03-12 23:00:00,60.486397,26.546460,94.283255
2,2022-03-12T23:59:35Z,23575.0,5067.0,2.0,2022-03-12 23:00:00,60.486397,26.546460,94.283255
3,2022-03-12T23:59:45Z,23576.0,5054.0,69.0,2022-03-12 23:00:00,60.514416,26.926898,115.104429
4,2022-03-12T23:59:45Z,23576.0,5071.0,3.0,2022-03-12 23:00:00,60.514416,26.926898,115.104429
...,...,...,...,...,...,...,...,...
6331,2022-03-13T01:32:05Z,23142.0,5067.0,1.0,2022-03-13 01:00:00,60.736118,25.448445,66.627738
6332,2022-03-13T01:32:05Z,23142.0,5056.0,107.0,2022-03-13 01:00:00,60.736118,25.448445,66.627738
6333,2022-03-13T01:32:05Z,23142.0,5057.0,107.0,2022-03-13 01:00:00,60.736118,25.448445,66.627738
6334,2022-03-13T01:32:05Z,23142.0,5071.0,1.0,2022-03-13 01:00:00,60.736118,25.448445,66.627738


In [25]:
df_tms_core = df_tms_hour[df_tms_hour.distance <= 20]

In [26]:
df_tms_core = df_tms_core.reset_index(drop=True)

In [27]:
df_tms_core = df_tms_core.drop(['distance'], axis=1)


In [28]:
logging.info(df_tms_core)

## Joinen der TMS Daten mit Wetter Daten

In [29]:
#import os, types
#import pandas as pd
#from botocore.client import Config
#import ibm_boto3

#def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
#client_8a835f62b295423a8d619b0d9819f636 = ibm_boto3.client(service_name='s3',
#    ibm_api_key_id='p3NvEesBPY7OsF2FSeaecnxTbsWpkOYYgdTVcwIk4CT2',
#    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
#    config=Config(signature_version='oauth'),
#    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

#body = client_8a835f62b295423a8d619b0d9819f636.get_object(Bucket='datentransformationtms-donotdelete-pr-ni7kxgngrqxfkp',Key='weather_data_cleaned.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
#if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

#weather = pd.read_csv(body)
#weather.head()


In [30]:
#tms_joined_weather = pd.merge(df_tms_core, weather, left_on="timestamp", right_on="timestamp", how="inner")

In [31]:
#tms_joined_weather = tms_joined_weather.reset_index(drop=True)
#tms_joined_weather.head()

In [32]:
#tms_joined_weather.shape

In [33]:
#df_tms_core.shape

# Csv ins Projekt speichern

In [34]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='f3fc764f-0a2b-474c-bb91-b082febf03fe', project_access_token='p-9e1a4b5cecba76c5bdba1e5660962be8e659a9ec')
pc = project.project_context


project.save_data(data=df_tms_core.to_csv(index=False),file_name=str(dateloading)+"-only_TMS_hour-"+str(filename_starttime)+"-"+str(filename_endtime)+".csv",overwrite=True)

{'file_name': '2022-03-13-only_TMS_hour-0-1.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'datentransformationtms-donotdelete-pr-ni7kxgngrqxfkp',
 'asset_id': '5904f737-a23e-468c-9883-3a14211bcd76'}